# Building MOP-Terphenyl Polymers using mbuild

Here we use `mbuild` to read in a SMILES string of a terphenyl monomer and build an n-residue polymer from the monomer.

In [1]:
import mbuild as mb
import numpy as np
import subprocess
from mbuild.lib.recipes.polymer import Polymer

First we get the MOP-terphenyl monomer from a smiles string.

In [2]:
comp = mb.load('C(=O)c1ccc(c2c(cc(OC)cc2)(c3cccc(c3)[C@@H](N)C))cc1', smiles = True, name="MOP")

I output all the indexes of hydrogen atoms because we will uses these indexes for extending the polymer later on.

In [3]:
for i, atom in enumerate(comp):
    if atom.name == "H":
        print(i, atom)

25 <H pos=([-0.2453 -0.5566  0.0782]), 1 bonds, id: 139860244202544>
26 <H pos=([-0.1188 -0.3599  0.1631]), 1 bonds, id: 139860244202784>
27 <H pos=([-0.0695 -0.1196  0.1639]), 1 bonds, id: 139860244203024>
28 <H pos=([ 0.0728  0.3003 -0.0153]), 1 bonds, id: 139860244203264>
29 <H pos=([-0.1463  0.6895  0.0515]), 1 bonds, id: 139860244203504>
30 <H pos=([-0.2465  0.5765  0.1501]), 1 bonds, id: 139860244203744>
31 <H pos=([-0.2739  0.5909 -0.0294]), 1 bonds, id: 139860244203984>
32 <H pos=([-0.3456  0.3696  0.0561]), 1 bonds, id: 139860244204224>
33 <H pos=([-0.3889  0.13    0.0462]), 1 bonds, id: 139860244204464>
34 <H pos=([-0.0317 -0.0545 -0.2148]), 1 bonds, id: 139860244204704>
35 <H pos=([ 0.1565 -0.2054 -0.2646]), 1 bonds, id: 139860244204944>
36 <H pos=([ 0.3579 -0.205  -0.1237]), 1 bonds, id: 139860244205184>
37 <H pos=([0.1828 0.1017 0.1242]), 1 bonds, id: 139860244205424>
38 <H pos=([0.3972 0.0289 0.1781]), 1 bonds, id: 139860244205472>
39 <H pos=([ 0.4395 -0.2537  0.0963]), 1

`mbuild` comes with a nice tool to visualize Compounds built into jupyter-notebooks. Using the object from `py3Dmol` we can coloro the atoms to identify the indices needed to make substiutions when building the polymer.

In [4]:
view = comp.visualize(show_ports=True)
style = {
                "stick": {"radius": 0.2, "color": "grey"},
                "sphere": {"scale": 0.3, "color" : "black"},
    }
view.setStyle({'model': -1, 'serial':26},style)
view.setStyle({'model': -1, 'serial':41},style)

/home/tfobe/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.3002 -0.4911  0.0117]), 3 bonds, id: 139860244273520> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <O pos=([-0.3879 -0.5407 -0.0639]), 1 bonds, id: 139860244273616> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.2717 -0.3465  0.0106]), 3 bonds, id: 139860244274288> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.1735 -0.2945  0.0961]), 3 bonds, id: 139860244274432>

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

We also make the two capping compounds using SMILES strings:

In [5]:
cap_o = mb.load('CO', smiles = True)
cap_o.visualize()

/home/tfobe/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.0357 -0.0032 -0.0017]), 4 bonds, id: 139860242864784> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <O pos=([ 0.0958 -0.0437 -0.0266]), 2 bonds, id: 139860242864880> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <H pos=([-0.0539  0.0029  0.1078]), 1 bonds, id: 139860242865024> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <H pos=([-0.1055 -0.0774 -0.0454]), 1 bonds, id: 139860242865168>

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [6]:
cap_n = mb.load('CC(C)(C)OC=O', smiles = True)
cap_n.visualize()

/home/tfobe/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.077   0.152  -0.0536]), 4 bonds, id: 139860108274032> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.0119  0.0207 -0.0076]), 4 bonds, id: 139860108274128> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.0768 -0.0231  0.1251]), 4 bonds, id: 139860108274272> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.0333 -0.0863 -0.1164]), 4 bonds, id: 139860242840304>

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Here we use the `Polymer` object to build a hexamer from the molecules we built in the previous cells. `Polymer.add_monomer()` is used to add the monomers to the polymer object. `Polymer.add_end_groups()` adds the capping groups to the object with `"head"` and `"tail"` labels for the each end of the polymer. `replace = True` will replace the specified atoms with the next residue. `indices` is used to specify which atom will be replaced in each group. Finally, when we call `Polymer.build()`, the polymer is built with the specified `n` residues and the provided capping residues. `sequence` is used if multiple monomers are provided.

In [7]:
chain = Polymer()
chain.add_monomer(compound=comp,
                  indices=[25, 40],
                  separation=.15,
                  replace=True,
                  # orientation = [[0,-1,0],[1,0,0]]
                 )
chain.add_end_groups(compound = cap_o,
                     index = -1,
                     separation=0.15,
                     label="head",
                     duplicate = False
                    )

chain.add_end_groups(compound = cap_n,
                     index = -1,
                     separation=0.15,
                     label="tail",
                     duplicate = False
                    )

chain.build(n=6, sequence='A')

In [8]:
chain.visualize()

/home/tfobe/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.3002 -0.4911  0.0117]), 3 bonds, id: 139860108323952> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <O pos=([-0.3879 -0.5407 -0.0639]), 1 bonds, id: 139860108323280> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.2717 -0.3465  0.0106]), 3 bonds, id: 139860108322272> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.1735 -0.2945  0.0961]), 3 bonds, id: 139860108320928>

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [9]:
for p in chain.particles():
    print(p)

<C pos=([-0.3002 -0.4911  0.0117]), 3 bonds, id: 139860108078864>
<O pos=([-0.3879 -0.5407 -0.0639]), 1 bonds, id: 139860108116320>
<C pos=([-0.2717 -0.3465  0.0106]), 3 bonds, id: 139860108116416>
<C pos=([-0.1735 -0.2945  0.0961]), 3 bonds, id: 139860108116560>
<C pos=([-0.145  -0.1577  0.0963]), 3 bonds, id: 139860108116704>
<C pos=([-0.2138 -0.0707  0.0101]), 3 bonds, id: 139860108116848>
<C pos=([-0.1831  0.0744  0.0093]), 3 bonds, id: 139860108116992>
<C pos=([-0.0513  0.1234 -0.0091]), 3 bonds, id: 139860108117136>
<C pos=([-0.0275  0.2617 -0.0011]), 3 bonds, id: 139860108117280>
<C pos=([-0.1319  0.3517  0.0222]), 3 bonds, id: 139860108117424>
<O pos=([-0.1017  0.4885  0.0285]), 2 bonds, id: 139860108117568>
<C pos=([-0.1977  0.5912  0.0514]), 4 bonds, id: 139860108117712>
<C pos=([-0.2626  0.3026  0.038 ]), 3 bonds, id: 139860108117856>
<C pos=([-0.2876  0.1655  0.0315]), 3 bonds, id: 139860108118000>
<C pos=([ 0.0623  0.0335 -0.0408]), 3 bonds, id: 139860108118144>
<C pos=([ 

In [10]:
# no need for rotation of peptide bonds
#for bond in chain.bonds():
#    if bond[0].name == "N":
#        if bond[1].n_direct_bonds == 3:
#            print("Rotating bond:", bond, "by", np.pi/2)
#            chain.rotate_dihedral(bond, np.pi/2)
#            
#    if bond[1].name == "N":
#        if bond[0].n_direct_bonds == 3:
#            print("Rotating bond:", bond, "by", np.pi/2)
#            chain.rotate_dihedral(bond, np.pi/2)       

In [11]:
chain.visualize(show_ports=True)

/home/tfobe/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.3002 -0.4911  0.0117]), 3 bonds, id: 139860105874784> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <O pos=([-0.3879 -0.5407 -0.0639]), 1 bonds, id: 139860105871568> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.2717 -0.3465  0.0106]), 3 bonds, id: 139860105654384> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.1735 -0.2945  0.0961]), 3 bonds, id: 139860105654480>

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Next we add specific residue labels for the componenets of the `Polymer` object. Here we label the monomers as HEX and the capping moieties as CAP.

In [12]:
print(chain.labels)
for label in chain.labels["monomer"]:
    label.name = "HEX"
    print(label)
for label in chain.labels["Compound"]:
    label.name = "CAP"
    print(label)

OrderedDict([('monomer', [<Compound 44 particles, 46 bonds, non-periodic, id: 139860107999984>, <Compound 44 particles, 46 bonds, non-periodic, id: 139860108027072>, <Compound 44 particles, 46 bonds, non-periodic, id: 139860107884528>, <Compound 44 particles, 46 bonds, non-periodic, id: 139860107501520>, <Compound 44 particles, 46 bonds, non-periodic, id: 139860107625568>, <Compound 44 particles, 46 bonds, non-periodic, id: 139860107282608>]), ('monomer[0]', <Compound 44 particles, 46 bonds, non-periodic, id: 139860107999984>), ('monomer[1]', <Compound 44 particles, 46 bonds, non-periodic, id: 139860108027072>), ('monomer[2]', <Compound 44 particles, 46 bonds, non-periodic, id: 139860107884528>), ('monomer[3]', <Compound 44 particles, 46 bonds, non-periodic, id: 139860107501520>), ('monomer[4]', <Compound 44 particles, 46 bonds, non-periodic, id: 139860107625568>), ('monomer[5]', <Compound 44 particles, 46 bonds, non-periodic, id: 139860107282608>), ('Compound', [<Compound 5 particles,

We save these as a pdb file and provide the names of the residues to include in the file.

In [13]:
chain.save("mop_hexamer_mbuild.pdb", overwrite=True, residues=["HEX", "CAP"])

/home/tfobe/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.3002 -0.4911  0.0117]), 3 bonds, id: 139860108078864> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <O pos=([-0.3879 -0.5407 -0.0639]), 1 bonds, id: 139860108116320> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.2717 -0.3465  0.0106]), 3 bonds, id: 139860108116416> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.1735 -0.2945  0.0961]), 3 bonds, id: 139860108116560>

Using Gromacs I generate a new gro file with the correct residue labels. `mbuild` doesn't seem to label residues correctly when writing `.gro` files.

In [14]:
! gmx editconf -f mop_hexamer_mbuild.pdb -o mop_hexamer_mbuild.gro

                     :-) GROMACS - gmx editconf, 2022.4 (-:

Executable:   /usr/local/bin/gmx
Data prefix:  /usr/local
Working dir:  /home/tfobe/Research/heteropolymer_simulations/examples/build_polymer/mop
Command line:
  gmx editconf -f mop_hexamer_mbuild.pdb -o mop_hexamer_mbuild.gro

Note that major changes are planned in future for editconf, to improve usability and utility.
Read 285 atoms
Volume: 37.1134 nm^3, corresponds to roughly 16700 electrons
No velocities found

Back Off! I just backed up mop_hexamer_mbuild.gro to ./#mop_hexamer_mbuild.gro.1#

GROMACS reminds you: "Friends don't let friends use Berendsen!" (John Chodera (on Twitter))



Lastly, I use openBabel to generate a `.mol` file for use in the OpenFF parameter assignment workflow.

In [15]:
! obabel -ipdb mop_hexamer_mbuild.pdb -omol mop_hexamer_mbuild.mol -O mop_hexamer_mbuild.mol

*** Open Babel Error  in ReadMolecule
  ERROR: not a valid PDB file

1 molecule converted


# Building arbitrary n-mers of MOP monomer

Now we put it all together to generate a tetramer, hexamer and octamer PMP terphenyl foldamer:

In [16]:
n_residues = [2,3,4,6,8,10]
residue_labels = ["DIM", "TRI","TET", "HEX", "OCT", "DEC"]
file_names = ["mop_dimer_mbuild", "mop_trimer_mbuild", "mop_tetramer_mbuild", "mop_hexamer_mbuild", "mop_octamer_mbuild", "mop_decamer_mbuild"]
for i in range(len(n_residues)):
    # Generate polymer
    chain = Polymer()
    chain.add_monomer(compound=comp,
                      indices=[25, 40],
                      separation=.15,
                      replace=True,
                      # orientation = [[0,-1,0],[1,0,0]]
                     )
    chain.add_end_groups(compound = cap_o,
                         index = -1,
                         separation=0.15,
                         label="head",
                         duplicate = False
                        )

    chain.add_end_groups(compound = cap_n,
                         index = -1,
                         separation=0.15,
                         label="tail",
                         duplicate = False
                        )

    chain.build(n=n_residues[i], sequence='A')

    # Rotate peptide bond
    #for bond in chain.bonds():
    #    if bond[0].name == "N":
    #        if bond[1].n_direct_bonds == 3:
    #            print("Rotating bond:", bond, "by", np.pi/2)
    #            chain.rotate_dihedral(bond, np.pi/2)

    #    if bond[1].name == "N":
    #        if bond[0].n_direct_bonds == 3:
    #            print("Rotating bond:", bond, "by", np.pi/2)
    #            chain.rotate_dihedral(bond, np.pi/2)  


    # Relabel chains
    for label in chain.labels["monomer"]:
        label.name = residue_labels[i]
        print(label)
    for label in chain.labels["Compound"]:
        label.name = "CAP"
        print(label)
            
            
    chain.save(file_names[i] + ".pdb", overwrite=True, residues=[residue_labels[i], "CAP"])
    
    subprocess.run(["gmx", "editconf", "-f", file_names[i] + ".pdb", "-o", file_names[i]+ ".gro"])
    subprocess.run(["obabel", "-ipdb", file_names[i] + ".pdb", "-omol", file_names[i] + ".mol",  "-O", file_names[i] + ".mol"])
    subprocess.run(["renumber_pdb_atoms", "-f", file_names[i] + ".pdb", "-o", file_names[i] + "_renum.pdb"])

<DIM 44 particles, 46 bonds, non-periodic, id: 139860078397616>
<DIM 44 particles, 46 bonds, non-periodic, id: 139860078397568>
<CAP 5 particles, 4 bonds, non-periodic, id: 139860078796608>
<CAP 16 particles, 15 bonds, non-periodic, id: 139860078841520>
Note that major changes are planned in future for editconf, to improve usability and utility.
Read 109 atoms
Volume: 8.36704 nm^3, corresponds to roughly 3700 electrons
No velocities found


/home/tfobe/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.3002 -0.4911  0.0117]), 3 bonds, id: 139860078422864> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <O pos=([-0.3879 -0.5407 -0.0639]), 1 bonds, id: 139860078449712> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.2717 -0.3465  0.0106]), 3 bonds, id: 139860078449904> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.1735 -0.2945  0.0961]), 3 bonds, id: 139860078450048>

*** Open Babel Error  in OpenAndSetFormat
  Cannot open mop_dimer_mbuild.mol
1 molecule converted


<TRI 44 particles, 46 bonds, non-periodic, id: 139860079769488>
<TRI 44 particles, 46 bonds, non-periodic, id: 139860079769152>
<TRI 44 particles, 46 bonds, non-periodic, id: 139860077585552>
<CAP 5 particles, 4 bonds, non-periodic, id: 139860078226352>
<CAP 16 particles, 15 bonds, non-periodic, id: 139860079819504>
Note that major changes are planned in future for editconf, to improve usability and utility.
Read 153 atoms
Volume: 11.9203 nm^3, corresponds to roughly 5300 electrons
No velocities found


/home/tfobe/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.3002 -0.4911  0.0117]), 3 bonds, id: 139860108026496> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <O pos=([-0.3879 -0.5407 -0.0639]), 1 bonds, id: 139860077923680> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.2717 -0.3465  0.0106]), 3 bonds, id: 139860077923872> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.1735 -0.2945  0.0961]), 3 bonds, id: 139860077924016>

...

<DEC 44 particles, 46 bonds, non-periodic, id: 139860073375872>
<DEC 44 particles, 46 bonds, non-periodic, id: 139860073375824>
<DEC 44 particles, 46 bonds, non-periodic, id: 139860073192176>
<DEC 44 particles, 46 bonds, non-periodic, id: 139860073353024>
<DEC 44 particles, 46 bonds, non-periodic, id: 139860072985552>
<DEC 44 particles, 46 bonds, non-periodic, id: 139860072646688>
<DEC 44 particles, 46 bonds, non-periodic, id: 139860072799152>
<DEC 44 particles, 46 bonds, non-periodic, id: 139860072394512>
<DEC 44 particles, 46 bonds, non-periodic, id: 139860072563856>
<DEC 44 particles, 46 bonds, non-periodic, id: 139860072200416>
<CAP 5 particles, 4 bonds, non-periodic, id: 139860074037008>
<CAP 16 particles, 15 bonds, non-periodic, id: 139860073819776>


/home/tfobe/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.3002 -0.4911  0.0117]), 3 bonds, id: 139860073392928> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <O pos=([-0.3879 -0.5407 -0.0639]), 1 bonds, id: 139860073419776> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.2717 -0.3465  0.0106]), 3 bonds, id: 139860073419968> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.1735 -0.2945  0.0961]), 3 bonds, id: 139860073420112>


Back Off! I just backed up mop_decamer_mbuild.gro to ./#mop_decamer_mbuild.gro.1#

GROMACS reminds you: "Jede der Scherben spiegelt das Licht" (Wir sind Helden)



Note that major changes are planned in future for editconf, to improve usability and utility.
Read 461 atoms
Volume: 121.016 nm^3, corresponds to roughly 54400 electrons
No velocities found


*** Open Babel Error  in ReadMolecule
  ERROR: not a valid PDB file

1 molecule converted
